In [ ]:
import pandas as pd
import numpy as np
import wrangle
import matplotlib as plt

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

In [ ]:
# Acquiring the data using the wrangle
zillow_sql_query = wrangle.get_zillow_data()
# I am working with a copy of the SQL query to be able to access it faster
df = zillow_sql_query.copy()

In [ ]:
zillow_sql_query.shape, df.shape

In [ ]:
df.head().T

### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
round(df.describe().T, 2)

In [ ]:
df.info

In [ ]:
# what percentage of null values for every column.  weird that all my buildingclasstypeid
# are null after sql query, there were values before

df.isnull().sum()/len(df)*100

In [ ]:
df.isnull().sum()

In [ ]:
# df.hist()
# plt.tight_layout

In [ ]:
# How many nulls have in each row?
# ...as in... 11995 rows have 33 nulls?
df.isnull().sum(axis =1).value_counts()

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def missing_values_col_wise(df):
    '''
    Function that takes in a df and returns a list of attributes with the number and percent
    of missing values for that attribute.
    '''
    new_df = pd.DataFrame(df.isnull().sum())
    new_df['number_missing'] = df.isnull().sum()
    new_df['percent_missing'] = df.isnull().sum()/len(df)*100
    new_df.drop([0], axis=1, inplace=True)
    return new_df

In [ ]:
missing_cols = missing_values_col_wise(df)
missing_cols

### Takeaways, missing values: 
Drop all columns over 50% except...
- pool count: i think pool count is 0 unless otherwise specified
- fireplace count: same
- unit count...see below

### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def missing_values_row_wise(df):
    '''
    Function that takes in a df and returns a list of rows with the number and percent
    of missing values.
    '''
    new_df = pd.DataFrame(df.isnull().sum(axis =1).value_counts())
    new_df['percent_missing'] = new_df.index/len(df.columns)*100
    new_df['num_of_rows'] = df.isnull().sum(axis =1).value_counts()
    new_df.drop([0], axis=1, inplace=True)
    new_df.index.rename('num_cols_missing_from_row', inplace = True)
    return new_df

In [ ]:
missing_rows = missing_values_row_wise(df)
missing_rows

### Takeaways, missing values row-wise: let's drop everything above fifty percent...actually, first let's drop columns and repeat process.

## Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
df.unitcnt.value_counts()

Start by removing anything above 1.0 units

In [ ]:
df.bedroomcnt.value_counts()

In [ ]:
df.calculatedfinishedsquarefeet.hist()

In [ ]:
# look at value counts for 'unitcnt'

df.unitcnt.value_counts(dropna = False)

In [ ]:
# (df.calculatedfinishedsquarefeet/df.bedroomcnt).hist()

In [ ]:
# Restrict df to only properties that meet single unit criteria

single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
df = df[df.propertylandusetypeid.isin(single_use)]
    
    
# Restrict df to only those properties with at least 1 bath & bed and >350 sqft area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.calculatedfinishedsquarefeet>350)]

df.shape

In [ ]:
def handle_missing_values(df, prop_required_row, prop_required_col):
    ''' function which takes in a dataframe, proportion of non-null rows and columns
    (respectively) required to prevent the columns and rows being dropped:'''
    
    #drop columns with nulls
    threshold = int(prop_required_col * len(df.index)) # Require that many non-NA values.
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    
    #drop rows with nulls
    threshold = int(prop_required_row * len(df.columns)) # Require that many non-NA values.
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    
    
    return df

In [ ]:
df = handle_missing_values(df, .7, .5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.heatingorsystemdesc.value_counts(dropna=False)

In [ ]:
# use your judgement to remove certain columns which you don't need

cols_to_remove = ['id',
       'calculatedbathnbr', 'finishedsquarefeet12', 'heatingorsystemtypeid'
       ,'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc'
       ,'regionidcounty', 
        'censustractandblock', 'propertylandusedesc', 'unitcnt']

In [ ]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
df = remove_columns(df,cols_to_remove)
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.heatingorsystemdesc.value_counts(dropna = False)

In [ ]:
# not sure i agree that there are so many homes lacking heat in southern CA
# but i do agree the attribute should be dropped

df.drop(columns = 'heatingorsystemdesc', inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
# buildingqualitytypeid
# Overall assessment of condition of the building from 
# low number = best quality
# higher numbers = worse quality

df.buildingqualitytypeid.value_counts(dropna = False)

In [ ]:
missing_values_col_wise(df)

In [ ]:
# what is the median value of buildingqualitytypeid?

df.buildingqualitytypeid.median()

In [ ]:
# should I fill missing values for buildingqualitytypeid with median value?
df.buildingqualitytypeid.fillna(6.0, inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
# look at lot-size distribution
df.lotsizesquarefeet.hist(bins = 100)

In [ ]:
# median lot size?
df.lotsizesquarefeet.median()

In [ ]:
df.lotsizesquarefeet.fillna(7313,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# I am going to drop the rest of nulls 

df.dropna(inplace = True)

In [ ]:
df.shape

In [ ]:
df.head(5).T

In [ ]:
df.isnull().sum()

In [ ]:
# look at distributions for all columns

df.hist(figsize=(24, 10), bins=20)
plt.tight_layout;

In [ ]:
# Remove properties valued more than 5 million dollars

df = df[df.taxvaluedollarcnt < 5_000_000]
df.shape

In [1]:
# Function to read and wrangle data:

def wrangle_zillow():
    df = pd.read_csv('zillow_df.csv')
    
    # Restrict df to only properties that meet single unit use criteria
    single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
    df = df[df.propertylandusetypeid.isin(single_use)]
    
    # Restrict df to only those properties with at least 1 bath & bed and 350 sqft area
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull())\
            & (df.calculatedfinishedsquarefeet>350)]

    # Handle missing values i.e. drop columns and rows based on a threshold
    df = handle_missing_values(df,.7,.5
                              )
    
    # Add column for counties
    df['county'] = np.where(df.fips == 6037, 'Los_Angeles',
                           np.where(df.fips == 6059, 'Orange', 
                                   'Ventura'))    
    # drop columns not needed
    df = remove_columns(df, ['id',
       'calculatedbathnbr', 'finishedsquarefeet12', 'heatingorsystemtypeid'
       ,'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc'
       ,'regionidcounty', 
        'censustractandblock', 'propertylandusedesc', 'unitcnt'])


#     replace nulls in unitcnt with 1
#     df.unitcnt.fillna(1, inplace = True)
    
    # assume that since this is Southern CA, null means 'None' for heating system
#     df.heatingorsystemdesc.fillna('None', inplace = True)

    # actually, I'm not assuming this, and I am dropping the heatingsystem col
    df.drop(columns = 'heatingorsystemdesc', inplace = True)
    
    # replace nulls with median values for select columns
    df.lotsizesquarefeet.fillna(7313, inplace = True)
    df.buildingqualitytypeid.fillna(6.0, inplace = True)

    # Columns to look for outliers
    df = df[df.taxvaluedollarcnt < 5_000_000]
    df = df[df.calculatedfinishedsquarefeet < 8000]
    
    # Just to be sure we caught all nulls, drop them here
    df = df.dropna()
    
    return df

In [2]:
df = wrangle_zillow()
df.head()

NameError: name 'pd' is not defined

In [ ]:
df.shape

In [ ]:
df.transactiondate.max()

# RETURN TO THIS NOTEBOOK TO COMPLETE THE MALL DATA EXERCISES